In [9]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from ipywidgets import interact, widgets
from IPython.display import display
import os
import sys
import requests
from requests.auth import HTTPBasicAuth
from src.udf import make_read_data_url
import pandas as pd
import json

In [10]:

# GLOBAL VARIABLES
prod_run = True if ('prod' in os.popen('hostname').read().rstrip()[-4:]) else False # Checks if running on prod instance
hana_sys = 'q' # Can either be '0', 'q' or '1'
use_cache = True # If True, then intermediate results are stored/read on/from S3
writeback_to_hana = True
datapath = os.path.join(os.path.dirname(os.getcwd()), "data")
print(datapath)
if not os.path.exists(datapath):
    os.makedirs(datapath)
#/home/ubuntu/matspec_ai/data
##Define odata HANA settings
# Get username and password for HANA
HANA_username = ("%s" % os.popen('/opt/parameter-get user').read()).rstrip('\n')
if (hana_sys == '0'):
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh0').read()).rstrip('\n')
else:
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh1').read()).rstrip('\n')


/home/ubuntu/matspec_ai/data


In [11]:
# Read parameters

# Create URL to CV
if (hana_sys == '1'):
    main_url = 'https://hanadatalake.siemens.com/siemens/MATSPEC/AWS/'
else:
    main_url = 'https://hanadatalake-eh' + hana_sys + '.siemens.com/siemens/MATSPEC/AWS/'

xsodata_hana_to_py = 'ODATA_CONNECTIONS.xsodata'
cmd_manufacturer = '/V_CMD_DATA'


url_cmd = make_read_data_url(main_url, xsodata_hana_to_py,cmd_manufacturer)
result = requests.get(url=url_cmd, auth=HTTPBasicAuth(HANA_username, HANA_password), verify=False)
df_cmd = pd.read_json(json.dumps(result.json()['d']['results']), orient='list')
df_cmd.drop(columns = ['GenID','__metadata'], inplace = True)
df_cmd.head()

/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


COMPANYNAME_GU
0                       " OBAL PRESS, spol. s r.o. "
1           " SWMS Systemtechnik Ingenieurges. mbH "
2               " TERMOELEKTRO - NS " d.o.o.Gradiska
3  ""LS 78 MANTES EN YVELINES"" LOISIRS SPORTIFS ...
4                                        "A M C" SRL

In [12]:
choices = list(set(df_cmd.COMPANYNAME_GU.tolist()))
len(choices)

508933

In [13]:
import re

In [14]:
choices_clean = [re.sub("\"|\(|\)|\&|\?|\.|\-|\,", "",  name.lower()) for name in choices]
choices_clean = [name.title() for name in choices_clean]
choices_clean = list(set(choices_clean))
len(choices_clean)

504035

## Das ist Markdown

In [15]:
text = widgets.Text(
    value='',
    placeholder='Paste Name here!',
    description='Company',
    disabled=False
)
#display(text)

def callback(wdgt):
    #print(wdgt)
    #display(process.extract(wdgt.value, choices, scorer=fuzz.token_set_ratio))
    return process.extract(wdgt, choices_clean, scorer=fuzz.token_set_ratio)
   # display(wdgt.value)
#text.on_submit(callback)

In [16]:
interact(callback, wdgt='');

interactive(children=(Text(value='', description='wdgt'), Output()), _dom_classes=('widget-interact',))